In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movie-recommendation-system/movies.csv
/kaggle/input/movie-recommendation-system/ratings.csv


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

In [3]:
movies=pd.read_csv("/kaggle/input/movie-recommendation-system/movies.csv")
ratings=pd.read_csv("/kaggle/input/movie-recommendation-system/ratings.csv")

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [6]:
ratings.shape

(25000095, 4)

In [10]:


ratings['userId'] = ratings['userId'].astype('category')
ratings['movieId'] = ratings['movieId'].astype('category')
ratings['rating'] = ratings['rating'].astype('float32')

user_index = ratings['userId'].cat.codes
movie_index = ratings['movieId'].cat.codes

sparse_matrix = csr_matrix((ratings['rating'], (movie_index, user_index)))


sparse_matrix


<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 25000095 stored elements and shape (59047, 162541)>

In [11]:
from sklearn.neighbors import NearestNeighbors

# Use cosine distance for similarity between items
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(sparse_matrix)


NearestNeighbors(algorithm='brute', metric='cosine')

In [12]:
def get_similar_movies(movie_title, top_n=10):
    # Find the movie ID from its title
    movie = movies[movies['title'].str.lower() == movie_title.lower()]
    if movie.empty:
        print("❌ Movie not found.")
        return []

    movie_id = movie.iloc[0]['movieId']   # original movieId
    matrix_index = ratings['movieId'].cat.categories.get_loc(movie_id)  # encoded row index

    # Get nearest neighbors (KNN) for that movie row
    distances, indices = knn.kneighbors(sparse_matrix[matrix_index], n_neighbors=top_n + 1)

    # Prepare list of similar movie titles (skip the first one, it's the same movie)
    similar_movies = []
    for idx in indices.flatten()[1:]:
        similar_id = ratings['movieId'].cat.categories[idx]  # original movieId
        title = movies[movies['movieId'] == similar_id]['title'].values[0]
        similar_movies.append(title)

    return similar_movies
def movie_founder(movie_title,top_n=10  ):
    asa=movies.loc[movies["title"]==Title]["movieId"].values[0]
    bsa=movie_id_reverse_map[asa]
    dist,idd=knn.kneighbors(sparse_matrix[bsa],n_neighbors=top_n+1)
    ads=[]
    for i in idd.flatten()[1:]:
        sim_id=movie_id_map[i]
        asa=movies.loc[movies["movieId"]==sim_id]["title"].values[0]
        ads.append(asa)
    
    return ads   
    


In [13]:
get_similar_movies("My Name is Khan (2010)", top_n=10)


['Marriage Made in Heaven, A (Rab Ne Bana Di Jodi) (2008)',
 'Ghajini (2008)',
 'Like Stars on Earth (Taare Zameen Par) (2007)',
 'Don (2006)',
 'Kal Ho Naa Ho (2003)',
 'OMG Oh My God! (2012)',
 'Something Is Happening (Kuch Kuch Hota Hai) (1998)',
 'Dilwale Dulhania Le Jayenge (1995)',
 'Chak De India! (2007)',
 'Ra.One (2011)']

In [ ]:
import joblib

# 4. Define a filename for the model
filename = 'sparse_matrix.pkl'

# 5. Save the model to the file
joblib.dump(sparse_matrix, filename)

print(f"Model saved as '{filename}'")

In [ ]:
pip install scikit-surprise


In [ ]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(0, 10.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

trainset = data.build_full_trainset()
svd = SVD()
svd.fit(trainset)


In [ ]:
# Map of raw movieId to inner id used by Surprise
movie_inner_ids = {trainset.to_raw_iid(i): i for i in trainset.all_items()}

# Latent features for each movie
movie_vectors = {movie_id: svd.qi[inner_id] for movie_id, inner_id in movie_inner_ids.items()}
